In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import load_csv_data, predict_labels, create_csv_submission
import math

## Load data

In [3]:
y_train, input_train, ids_train = load_csv_data('train.csv', sub_sample=False)
print(y_train.shape)
print(input_train.shape)

(250000,)
(250000, 30)


## Standardize data

In [4]:
def standardize_train(x):
    ''' standardize training set
    '''
    centered_data = x - np.mean(x, axis=0)
    std_data = centered_data / np.std(x, axis=0)
    
    return std_data, np.mean(x, axis=0), np.std(x, axis=0)

def standardize_test(x, mean, std):
    ''' standardize test set with same values as training set
    '''
    return (x-mean)/std

In [5]:
# IDEE: on pourrait ausi tenter build_polynomial comme dans les déries si on est motivés

In [6]:
# Standardise data
# Careful to standardize the x_test with the mean and std of x_train
x_train, mean, std = standardize_train(input_train)
#x_test = standardize_test(input_test, mean, std)

In [7]:
print(x_train.shape)
#print(x_test.shape)

(250000, 30)


In [8]:
input_train

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

## MSE method GD and SGD from lab02
I put it here to use as reference for the function to implement.

Could try it too  --> will be less good than Ridge if find good lambda

In [39]:
# Define the parameters of the algorithm. (probably change afterwards)
max_iters = 5                                                    ############ MEGA LONG (baisser si jamais)
gamma = 0.7
batch_size = 1

# Initialization
w_initial = np.ones(x_train.shape[1])

In [54]:
#From lab02 correction
# MSE GD and SGD
def calculate_mse(e):
    """Calculate the mse for vector e."""
    return np.mean(e**2)/2

def compute_loss_mse(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mse or mae.
    """
    e = y - tx.dot(w)
    return calculate_mse(e)

def compute_gradient_mse(y, tx, w):
    """Compute the gradient."""
    err = y - tx.dot(w)
    #print(err)
    #print(len(err))
    #print(-tx.T.dot(err))
    grad = -tx.T.dot(err) / len(err)
    return grad, err

def gradient_descent_mse(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute loss, gradient
        grad, err = compute_gradient_mse(y, tx, w)
        loss = calculate_mse(err)
        # gradient w by descent update
        w = w - gamma * grad
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

def stochastic_gradient_descent(
        y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_gradient_mse(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            loss = compute_loss_mse(y, tx, w)
            # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}".format(bi=n_iter, ti=max_iters - 1, l=loss))
    return losses, ws

In [55]:
losses, ws = gradient_descent_mse(y_train, x_train, w_initial, max_iters, gamma)

Gradient Descent(0/9): loss=0.5, w0=0.15890072268734834, w1=-0.23350355904089673
Gradient Descent(1/9): loss=1.3151906775211355, w0=-0.12798293911466097, w1=0.05539970420684712
Gradient Descent(2/9): loss=51.21992783427874, w0=1.5545037905755277, w1=-1.6951184187302437
Gradient Descent(3/9): loss=2924.809070536689, w0=-11.051329809331301, w1=11.09869204845918
Gradient Descent(4/9): loss=168340.19612585776, w0=84.39306689457838, w1=-85.82113308277333
Gradient Descent(5/9): loss=9690293.438496064, w0=-639.7203788705657, w1=649.3098422008295
Gradient Descent(6/9): loss=557810954.067767, w0=4854.103153077977, w1=-4928.157753964412
Gradient Descent(7/9): loss=32109767876.289787, w0=-36828.015945113555, w1=37388.47513051421
Gradient Descent(8/9): loss=1848363117047.5952, w0=279417.80295701657, w1=-283671.50264545856
Gradient Descent(9/9): loss=106398969486965.27, w0=-2119966.7483497383, w1=2152238.427913745


## Should be useful

In [56]:
#besoin pour SGD, prise de lab02
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]      

In [57]:
# Celle du lab03, juste sans les degrés
def plot_train_test(train_errors, test_errors, lambdas):
    plt.semilogx(lambdas, train_errors, color='b', marker='*', label="Train error")
    plt.semilogx(lambdas, test_errors, color='r', marker='*', label="Test error")
    plt.xlabel("lambda")
    #plt.ylabel("RMSE")
    plt.ylabel("Loss")
    plt.title("Ridge regression for polynomial degree ")
    leg = plt.legend(loc=1, shadow=True)
    leg.draw_frame(False)
    #plt.savefig("ridge_regression")

# Methods to implement 
P.2 of project description

-  least_square
-  least_square_GD
-  least_square_SGD

-  ridge_regression
-  logistic_regression
-  reg_logistic_regression

All function should return (w, loss) <br>
Later add them in implementations.py

In [58]:
# Define the parameters of the algorithm. (probably change afterwards)
max_iters = 10                                          
gamma = 0.7
batch_size = 1

# Initialization
w_initial = np.zeros(x_train.shape[1])

## Least square 

In [63]:
def compute_loss_ls(y, tx, w):
    """Calculate the loss.
    """
    e = y - tx.dot(w)
    return 1/2*np.mean(e**2)

In [64]:
def least_squares(y, tx):
    """Compute the optimal w and the loss with least square technique"""
    """Required by project description"""
    a = tx.T.dot(tx) 
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)                #p.7 du cours least squares
    loss = compute_loss_ls(y, tx, w)                 #p.3 du cours least squares
    return w, loss

In [65]:
w_ls, loss_ls = least_squares(y_train, x_train)
print("w is {} \n of shape {}".format(w_ls, w_ls.shape))
print("\n loss of least squares is {}".format(loss_ls))

w is [ 2.93788275e-02 -2.52531475e-01 -2.54791125e-01 -3.03696825e-02
 -1.40144671e+00  2.95701639e-01 -1.07889471e+01  2.67880862e-01
 -2.44934960e-03 -3.28843215e+02 -1.82647888e-01  1.14039627e-01
  2.05045956e+01  6.38883723e+01 -3.18961843e-04 -1.80884298e-03
  6.29974773e+01 -4.48641621e-04  1.54379308e-03  1.21462700e-01
  3.95268639e-04 -6.33223479e-02 -2.06747092e-01 -1.16655759e-01
  9.86256238e-02  1.67907719e-01 -3.35146169e-02 -2.98358686e+00
 -5.36388110e+00  2.78492205e+02] 
 of shape (30,)

 loss of least squares is 0.3889523149372305


In [66]:
def compute_gradient_least_squares(y, tx, w):
    """Compute the gradient of least square."""
    '''
    print("y is {}".format(y.shape))
    print("tx is {}".format(tx.shape))
    print("w is {}".format(w.shape))
    a = tx.dot(w)
    print("a is {}".format(a.shape))
    '''
    err = y - w.dot(tx.T)#tx.dot(w)
    #print(err)
    #print(err.shape)
    grad = -(tx.T @ err) / len(err)          #p.5 du cours least squares
    return grad                  

In [67]:
#compute_gradient_least_square(y_train, x_train, w)
grad_ls = compute_gradient_least_squares(y_train, x_train, w_initial)
print("grad is {} \n of shape {}".format(grad_ls, grad_ls.shape))
#print("\nerr of least squares is {}".format(err))

grad is [-0.22700103  0.33357651  0.01334131 -0.18274659 -0.13445082 -0.18202497
 -0.13341468 -0.01162345  0.01451087 -0.14545203  0.18547229 -0.25794773
 -0.13416605 -0.22328862  0.00089534  0.0041789   0.03032475 -0.00143922
 -0.00391589 -0.02132456 -0.00709562 -0.12863626 -0.12676524 -0.14959835
 -0.14282589 -0.14282473 -0.13356652 -0.1340891  -0.13408034 -0.12747392] 
 of shape (30,)


In [74]:
def least_squares_GD(y, tx, w_initial, max_iters, gamma):
    """Gradient descent algorithm with least squares."""
    """Required by project description"""
    w = w_initial
    for n_iter in range(max_iters):
        # compute gradient and error
        grad = compute_gradient_least_squares(y, tx, w)
        print(grad)
        # gradient w by descent update
        w = w - (gamma * grad)
        # print(w)
        # calculate loss    
    loss = compute_loss_ls(y, tx, w)           
    return w, loss

In [75]:
w_ls_gd, loss_ls_gd = least_squares_GD(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_ls_gd, w_ls_gd.shape))
print("\n loss of least squares is {}".format(loss_ls_gd))

[-0.22700103  0.33357651  0.01334131 -0.18274659 -0.13445082 -0.18202497
 -0.13341468 -0.01162345  0.01451087 -0.14545203  0.18547229 -0.25794773
 -0.13416605 -0.22328862  0.00089534  0.0041789   0.03032475 -0.00143922
 -0.00391589 -0.02132456 -0.00709562 -0.12863626 -0.12676524 -0.14959835
 -0.14282589 -0.14282473 -0.13356652 -0.1340891  -0.13408034 -0.12747392]
[ 4.09833803e-01 -4.12718947e-01 -3.19436834e-02  1.00423868e+00
  1.19549526e+00  1.12224957e+00  1.19614187e+00 -5.84514813e-01
  4.80440258e-01  1.14057349e+00 -4.51196927e-02  7.56294840e-01
  1.19570789e+00  4.46125732e-01  1.01180311e-02  5.04438891e-03
  2.20402526e-01  2.18216906e-02 -6.63131742e-04  5.14281933e-01
  1.41272120e-02  1.09991388e+00  1.28921671e+00  1.07225450e+00
  1.02897974e+00  1.02898049e+00  1.20256092e+00  1.19575741e+00
  1.19575608e+00  1.19480454e+00]
[-2.40355247  2.50074018  0.39329383 -7.71054727 -9.19524218 -8.95951789
 -9.19385953  4.32670567 -3.67810119 -8.7040835  -0.49328688 -5.44884171

[-3.13529135e+21  3.18301931e+21  4.58992312e+20 -1.02025654e+22
 -1.21895064e+22 -1.18282529e+22 -1.21886894e+22  5.76812411e+21
 -4.88945583e+21 -1.15463078e+22 -7.44396479e+20 -7.13923197e+21
 -1.21896622e+22 -4.18523384e+21 -9.18936187e+19 -6.31912213e+19
 -3.60588682e+21 -1.96989098e+20  1.60861551e+19 -6.04634913e+21
 -1.07000217e+20 -1.08574300e+22 -1.26395686e+22 -1.02955419e+22
 -9.83276496e+21 -9.83281916e+21 -1.22528558e+22 -1.21896102e+22
 -1.21895457e+22 -1.18615230e+22]
[ 2.37877282e+22 -2.41498444e+22 -3.48241460e+21  7.74077516e+22
  9.24828456e+22  8.97419840e+22  9.24766466e+22 -4.37632595e+22
  3.70967268e+22  8.76028421e+22  5.64780084e+21  5.41659741e+22
  9.24840276e+22  3.17537332e+22  6.97204880e+20  4.79437293e+20
  2.73581770e+22  1.49457343e+21 -1.22047058e+20  4.58741769e+22
  8.11819958e+20  8.23762661e+22  9.58975063e+22  7.81131720e+22
  7.46020432e+22  7.46024544e+22  9.29634830e+22  9.24836334e+22
  9.24831439e+22  8.99944069e+22]
[-1.80479564e+23  1.83

[-7.20990115e+40  7.31965613e+40  1.05549655e+40 -2.34617712e+41
 -2.80309312e+41 -2.72001945e+41 -2.80290524e+41  1.32643509e+41
 -1.12437695e+41 -2.65518349e+41 -1.71181062e+40 -1.64173441e+41
 -2.80312895e+41 -9.62434393e+40 -2.11318131e+39 -1.45314233e+39
 -8.29208026e+40 -4.52995198e+39  3.69916462e+38 -1.39041558e+41
 -2.46057193e+39 -2.49676947e+41 -2.90658920e+41 -2.36755794e+41
 -2.26113798e+41 -2.26115044e+41 -2.81766092e+41 -2.80311700e+41
 -2.80310217e+41 -2.72767021e+41]
[ 5.47021472e+41 -5.55348678e+41 -8.00814414e+40  1.78006499e+42
  2.12673114e+42  2.06370242e+42  2.12658859e+42 -1.00637784e+42
  8.53074573e+41  2.01451081e+42  1.29876561e+41  1.24559818e+42
  2.12675832e+42  7.30207347e+41  1.60328904e+40  1.10251173e+40
  6.29127343e+41  3.43691397e+40 -2.80658838e+39  1.05492040e+42
  1.86685733e+40  1.89432072e+42  2.20525452e+42  1.79628681e+42
  1.71554505e+42  1.71555451e+42  2.13778385e+42  2.12674926e+42
  2.12673800e+42  2.06950712e+42]
w is [-3.38323072e+41 

In [ ]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """Stochastic Gradient Descent algorithm with least squares."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad = compute_gradient_least_squares(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
    loss = compute_loss_ls(y, tx, w)                                 #TO CHECK p.3 least squares
    return w, loss

In [ ]:
w_ls_sgd, loss_ls_sgd = least_squares_SGD(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w_ls_sgd, w_ls_sgd.shape))
print("\n loss of least squares is {}".format(loss_ls_sgd))

## Ridge regression

In [76]:
# Should do grid search  TO DO
lambda_ = 0.01

In [79]:
##################
# Je suis un peu confuse. Est ce que ce qu'il veut c'est ça ou bien ce que j'ai fait aux fonctions ridge_GD et ridge_SGD ?? 
# ridge_GD et ridge_SGD sont juste en-dessous
def ridge_regression(y, tx, lambda_):
    """Ridge regression."""
    """Required by project description"""
    lambd = 2 * tx.shape[0] * lambda_
    a = tx.T.dot(tx) + lambd * np.identity(tx.shape[1])
    b = tx.T.dot(y) 
    w = np.linalg.solve(a, b) 
    err = y - tx.dot(w)
    loss = (1/2) * np.mean(err**2) + lambda_ * (np.linalg.norm(w,2))**2   #TO CHECK p.3 ridge regression
    return w, loss

In [80]:
w_rreg, loss_rreg = ridge_regression(y_train, x_train, lambda_)
print("w is {} \n of shape {}".format(w_rreg, w_rreg.shape))
print("\n loss of ridge regression is {}".format(loss_rreg))

w is [ 0.03572402 -0.24077855 -0.21610016 -0.01054923 -0.01919922  0.34814992
 -0.03018983  0.23504977 -0.01013674 -0.00087247 -0.15674991  0.11455458
 -0.02161131  0.18503225 -0.00076269 -0.00130455  0.23850622 -0.00086979
  0.00249252  0.10383837  0.00113401 -0.06198121 -0.14991608  0.03645409
  0.04367213  0.0436772  -0.02328488 -0.02309077 -0.0234825  -0.09703028] 
 of shape (30,)

 loss of ridge regression is 0.39491577663267774


In [81]:
def compute_gradient_ridge(y, tx, w, lambda_):
    """Not required specifically""" 
    err = y - tx.dot(w)
    grad = -tx.T.dot(err) / len(err) + (2 * lambda_* w)
    return grad, err

In [82]:
def ridge_GD(y, tx, initial_w, max_iters, gamma, lambda_):
    """Gradient descent algorithm with Ridge."""
    """Not required specifically""" 
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and error
        grad, _ = compute_gradient_ridge(y, tx, w, lambda_)
        #print("grad is {} \n of shape {}".format(grad, grad.shape))
        #print("\n err of least squares is {}".format(err))
        # gradient w by descent update
        w = w - gamma * grad
        # calculate loss
    loss = 1/2*np.mean((y - tx.dot(w))**2)            
    return w, loss

In [83]:
w, loss = ridge_GD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
print("w is {} \n of shape {}".format(w, w.shape))
print("\n loss of ridge_GD is {}".format(loss))

w is [-3.70387313e+41  3.76025650e+41  5.42230084e+40 -1.20527899e+42
 -1.44000606e+42 -1.39732941e+42 -1.43990953e+42  6.81416734e+41
 -5.77615350e+41 -1.36402186e+42 -8.79391996e+40 -8.43392417e+41
 -1.44002446e+42 -4.94422159e+41 -1.08558430e+40 -7.46508827e+39
 -4.25981059e+41 -2.32712864e+40  1.90033624e+39 -7.14284818e+41
 -1.26404594e+40 -1.28264135e+42 -1.49317410e+42 -1.21626276e+42
 -1.16159265e+42 -1.16159906e+42 -1.44748983e+42 -1.44001832e+42
 -1.44001070e+42 -1.40125977e+42] 
 of shape (30,)

 loss of ridge_GD is 1.869569596937695e+86


In [ ]:
def ridge_SGD(y, tx, initial_w, max_iters, gamma, lambda_):
    """Stochastic Gradient Descent algorithm with least squares."""
    """Not required specifically""" #Implicitely requested I think
    
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_gradient_ridge(y_batch, tx_batch, w, lambda_)    
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
    loss = np.mean((y - tx.dot(w))**2)/2 + lambda_ * (np.linalg.norm(w,2))**2
    return w, loss

In [ ]:
w, loss = ridge_SGD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
print("w is {} \n of shape {}".format(w, w.shape))
print("\n loss of ridge SGD is {}".format(loss))

In [ ]:
lambdas = np.logspace(-4, -0.05, 15)

rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    w, loss_tr = ridge_SGD(y_train, x_train, w_initial, max_iters, gamma, lambda_)
    rmse_tr.append(np.sqrt(2*loss_tr))
    loss_te = (1/2) * np.mean((y_test - x_test.dot(w))**2) + lambda_ * (np.linalg.norm(w,2))**2
    rmse_te.append(np.sqrt(2*loss_te))

In [ ]:
# Grid search to choose lambda
plot_train_test(rmse_tr, rmse_te, lambdas)

## Logistic regression
Debugged but not sure from here: Il y aura probablement une correction de labo pour améliorer / re-vérifier les fonctions de logistic regression

In [ ]:
def sigmoid(tx, w):
    """Compute sigmoid function"""
    z = np.array(np.exp(-tx.dot(w)))
    return 1./(1 + z)

value = sigmoid(x_train, w_initial)
print(value)

In [ ]:
def compute_logreg_loss(y, tx, w):  #np.log parce que math.log fonctionne pas.. J'ai toujours pas compris pourquoi.
    """Compute error and gradient of logistic regression"""
    sig = sigmoid(tx, w)
    loss = sum(-y * np.log(sig) - (1-y) * np.log(1-sig))/len(y)
    return loss

In [ ]:
loss = compute_logreg_loss(y_train, x_train, w_initial)
print("loss is {}".format(loss))

In [ ]:
def compute_logreg_grad(y, tx, w):
    """Compute error and gradient of logistic regression""" 
    sig = sigmoid(tx, w)
    err  = sig - y
    grad = tx.T.dot(err)/len(y)
    return grad

In [ ]:
grad = compute_logreg_grad(y_train, x_train, w_initial)
print("grad is {} \n of shape {}".format(grad, grad.shape))

In [ ]:
def logistic_regression(y, tx, initial_w, max_iters, gamma): #SGD  (GD easy to implement from here)
    """Stochastic Gradient Descent algorithm with logistic regression."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient
            grad = compute_logreg_grad(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # compute a stochastic loss
            compute_logreg_loss(y_batch, tx_batch, w)
    return w, loss

In [ ]:
w, loss = logistic_regression(y_train, x_train, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w, w.shape))
print("\n loss of ridge_SGD is {}".format(loss))

## Regularized Logistic Regression

In [ ]:
def compute_logreg_reg_loss(y, tx, w, lambda_):
    """Compute error and gradient of logistic regression"""
    reg = ( lambda_/(2*len(y)) ) * sum(w**2)
    loss = compute_logreg_loss(y, tx, w) + reg
    return loss

In [ ]:
loss = compute_logreg_reg_loss(y_train, x_train, w_initial, lambda_)
print("loss is {}".format(loss))

In [ ]:
def compute_logreg_reg_grad(y, tx, w, lambda_):
    """Compute error and gradient of logistic regression"""     
    grad = compute_logreg_grad(y, tx, w) 
    reg = (lambda_/len(y)) * w[1:]
    grad[1:] = grad[1:] + reg            
    return grad

In [ ]:
grad = compute_logreg_reg_grad(y_train, x_train, w_initial, lambda_)
print("grad is {} \n of shape {}".format(grad, grad.shape))

In [ ]:
###########
# La loss de ridge SGD me semble particulièrement haute. A vérifier avec les plots (et/ou code).

def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    """Stochastic Gradient Descent algorithm with REGULARIZED logistic regression."""
    """Required by project description"""
    w = initial_w

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient
            grad = compute_logreg_reg_grad(y_batch, tx_batch, w, lambda_)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # compute a stochastic loss
            loss = compute_logreg_reg_loss(y_batch, tx_batch, w, lambda_)
    return w, loss

In [ ]:
w, loss = reg_logistic_regression(y_train, x_train, lambda_, w_initial, max_iters, gamma)
print("w is {} \n of shape {}".format(w, w.shape))
print("\n loss of ridge_SGD is {}".format(loss))

# Prediction de chaque méthode

In [ ]:
#y_pred = predict_labels(weights, data)

## Choisir méthode:
(D'après moi mais hésitez pas à faire autrement si vous avez une meilleure idée)
- Pour chaque méthode: faire un plot de test_error (=loss) pour un grand nombre d'iteration (assez pour que on voit que l'erreur commence à remonter si possible). 
- Garder la valeur de n iter ou la test error était minimale (au cas où: risque d'être très long pour ridge si lambda_ trop grand)
- Comparer les loss de chaque méthode à leur n_iter optimale
- La meilleure est celle ou c'est le plus faible

En plus comme ça on aura des beau plots et des arguments pour le rapport.